<a href="https://colab.research.google.com/github/jackliu333/object_detection_using_tensorflow/blob/main/FoodBank_Training_Customised_Image_Detection_Model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.9
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

In [ ]:
# Codes adapted from https://github.com/nicknochnack/TFODCourse/blob/main/2.%20Training%20and%20Detection.ipynb
# Images sampled from https://github.com/jackliu333/object_detection_using_tensorflow, with bounding boxes drawn manually for each object
# To run using Google Colab; switch runtime type to GPU for speedup
import numpy as np
import random
import tensorflow as tf

SEED = 123
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
# import tensorflow as tf
tf.version.VERSION

# Configuration parameters

Including paths and training hyper-parameters

In [ ]:
import os
print(f"Current OS: {os.name}") 

In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

# SSD has good tradeoff between speed and accuracy; can switch to other pretrained model
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

# TF official script to encode training data to tf record format
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'

# Mapping dictionary between label and integer id
LABEL_MAP_NAME = 'label_map.pbtxt'

# Define a list of folder paths to be created (if needed) and used later
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    # bounding box annotation
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
}

training_files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
# create folder paths if not exist
for p in paths.values():
    if not os.path.exists(p):
        !mkdir -p {p}

# Set up TF model training logistics

### Download TF model training utility scripts from TF model zoo

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'objection_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

### Install TF object detection library

May hit some error during installation, can be ignored, as long as it returns "OK" in the last line.

In [ ]:
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 


In [ ]:
# Verify installation
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
!python {VERIFICATION_SCRIPT}

In [ ]:
# test if TF object detection library could be loaded
import object_detection

In [ ]:
# reinstall TF and restart runtime
# !pip install tensorflow==2.7.0

### Download and decompress TF pretrained model

In [ ]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

### Download training images

In [ ]:
import shutil

if os.path.exists('object_detection_using_tensorflow'):
    shutil.rmtree('object_detection_using_tensorflow')

!git clone https://github.com/jackliu333/object_detection_using_tensorflow.git

### Create label map

In [ ]:
# labels = [{'name':'Apple', 'id':1},
#           {'name':'Avocado', 'id':2},
#           {'name':'Banana', 'id':3},
#           {'name':'Cabbage', 'id':4},
#           {'name':'Carrot', 'id':5},
#           {'name':'Potato', 'id':6}]
# labels = [{'name':'Canned_Sardines_and_Mackerel', 'id':1}, 
#           {'name':'Canned_Baked_Beans', 'id':2}, 
#           {'name':'Biscuits', 'id':3},
#           {'name':'Halal', 'id':4},
#           {'name':'Kitkat', 'id':5},
#           {'name':'Wafers', 'id':6}]

labels = [{'name':'cookies_1g-99g', 'id':1}, 
          {'name':'cookies_1kg-1.99kg', 'id':2}, 
          {'name':'cookies_100g-199g', 'id':3},
          {'name':'cookies_200g-299g', 'id':4},
          {'name':'crackers_100g-199g', 'id':5},
          {'name':'crackers_200g-299g', 'id':6},
          {'name':'crackers_300g-399g', 'id':7},
          {'name':'crackers_400g-499g', 'id':8},
          {'name':'sardines_100g-199g', 'id':9},
          {'name':'sardines_400g-499g', 'id':10},
          {'name':'Baked beans_400-499g', 'id':11},
          {'name':'cookies_300g-399g', 'id':12},
          {'name':'cookies_400g-499g', 'id':13},
          {'name':'cookies_500g-599g', 'id':14},
          {'name':'cookies_600g-699g', 'id':15},
          {'name':'crackers_1g-99g', 'id':16},
          {'name':'crackers_800g-899g', 'id':17},
          {'name':'halal', 'id':18}]

# labels = [{'name':'cookies_1g-99g', 'id':1}, 
#           {'name':'cookies_1kg-1.99kg', 'id':2},
#           {'name':'cookies_100g-199g', 'id':3},
#           {'name':'cookies_200g-299g', 'id':4},
#           {'name':'crackers_100g-199g', 'id':5},
#           {'name':'crackers_200g-299g', 'id':6},
#           {'name':'crackers_300g-399g', 'id':7},
#           {'name':'crackers_400g-499g', 'id':8}]

# labels = [{'name':'Baked beans_400-499g', 'id':1}]

with open(training_files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

### Split into train test folders

In [ ]:
tmp_folders = ['train', 'test']

for i in tmp_folders:
    if os.path.exists(os.path.join(paths['IMAGE_PATH'], i)):
        shutil.rmtree(os.path.join(paths['IMAGE_PATH'], i))
        !mkdir -p {os.path.join(paths['IMAGE_PATH'], i)}
    else:
        !mkdir -p {os.path.join(paths['IMAGE_PATH'], i)}

In [ ]:
import shutil

for i in range(len(labels)):
    # print(labels[i]['name'])
    from_path = os.path.join('object_detection_using_tensorflow','images5',labels[i]['name'])
    # print(from_path)

    # get unique file names
    tmp_files = os.listdir(from_path)
    tmp_names = []
    tmp_file_types = []
    for tmp_file in tmp_files:
        tmp_name = os.path.splitext(tmp_file)[0]
        tmp_file_type = os.path.splitext(tmp_file)[1]
        tmp_names.append(tmp_name)
        tmp_file_types.append(tmp_file_type)
    tmp_names = list(set(tmp_names))
    tmp_names = [i for i in tmp_names if i != '.DS_Store']
    tmp_file_types = list(set(tmp_file_types))
    tmp_file_types = [i for i in tmp_file_types if len(i) != 0]
    # random shuffle the files
    random.shuffle(tmp_names)
    
    # training and test files
    tmp_names_train = tmp_names[0:int(len(tmp_names)*0.9)]
    tmp_names_test = [i for i in tmp_names if i not in tmp_names_train]

    # move into respective target folders
    for tmp_name in tmp_names_train:
        for tmp_file_type in tmp_file_types:
            tmp_name_full = tmp_name + tmp_file_type
            if(os.path.exists(os.path.join(from_path, tmp_name_full))):
                shutil.copy(os.path.join(from_path, tmp_name_full), \
                            os.path.join(paths['IMAGE_PATH'], "train"))

    for tmp_name in tmp_names_test:
        for tmp_file_type in tmp_file_types:
            tmp_name_full = tmp_name + tmp_file_type
            if(os.path.exists(os.path.join(from_path, tmp_name_full))):    
                shutil.copy(os.path.join(from_path, tmp_name_full), \
                            os.path.join(paths['IMAGE_PATH'], "test"))

In [ ]:
# from_path
# os.listdir(from_path)

### Create TF Record

In [ ]:
# download conversion script
if not os.path.exists(training_files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

In [ ]:
# convert to TF record format
!python {training_files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {training_files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {training_files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {training_files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

### Copy model configuration file to training folder

In [ ]:
!cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {paths['CHECKPOINT_PATH']}

### Update configuration file for transfer learning

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

# Read current configuration file
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(training_files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

# Update based on new labels
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= training_files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = training_files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

# # Add more data augmentation
# upd_dict = {'data_augmentation_options':{'random_rgb_to_gray' : {}}}
# # pipeline_config.train_config = upd_dict
# pipeline_config.train_config.data_augmentation_options.random_rgb_to_gray = []
# # Load the pipeline config as a dictionary
# pipeline_config_dict = config_util.get_configs_from_pipeline_file(pipeline_config)
# # update dict entry

# pipeline_config_dict["train_config"].update(upd_dict)
# # Convert the pipeline dict back to a protobuf object
# pipeline_config = config_util.create_pipeline_proto_from_configs(pipeline_config_dict)

# Write to configuration file
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(training_files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# Model training

In [ ]:
# fix opencv-python version issue 
!pip uninstall opencv-python-headless 
!pip install opencv-python-headless==4.1.2.30

In [ ]:
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
# # import fileinput
# file_path = '/usr/local/lib/python3.7/dist-packages/official/modeling/optimization/optimizer_factory.py'
# # fields = {"adamw_experimental": "tf.keras.optimizers.Adam"}

# with open(file_path, 'r') as file:
  
#     data = file.read()
#     data = data.replace('tf.keras.optimizers.experimental.AdamW', 'tf.keras.optimizers.AdamW')
 
# with open(file_path, 'w') as file:
  
#     # Writing the replaced data in our
#     # text file
#     file.write(data)

# # print(test)

In [ ]:
training_steps = 30000
check_point = 'ckpt-31'
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],training_files['PIPELINE_CONFIG'],training_steps)
!{command}

In [ ]:
training_files['PIPELINE_CONFIG']

In [ ]:
import tensorflow as tf
tf.version.VERSION

In [ ]:
# with open('/usr/local/lib/python3.7/dist-packages/official/modeling/optimization/optimizer_factory.py', 'r') as f:
    # print(f.read())

# (Optional) Model evaluation

In [ ]:
# # stop evaluation if needed
# command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])
# !{command}

# Predicting object category in image

### Load trained model from checkpoint

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(training_files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], check_point)).expect_partial()

# @tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [ ]:
paths['CHECKPOINT_PATH']

### Detect objects in image

In [ ]:
import cv2 
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 10]

# Randomly select an image to be detected
# tmp_img = random.choice([file for file in os.listdir(os.path.join(paths['IMAGE_PATH'], 
#                                               'test')) if file.endswith(".JPG")])
# IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', tmp_img)
tmp_img = random.choice([file for file in os.listdir(os.path.join(paths['IMAGE_PATH'], 
                                              'train')) if file.endswith(".png")])
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'train', tmp_img)
IMAGE_PATH = './crackers_100-199g_H_1.jpg'

category_index = label_map_util.create_category_index_from_labelmap(training_files['LABELMAP'])

img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=2,
            min_score_thresh=.7,
            agnostic_mode=False,
            line_thickness=8)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
print("detected class:")
print(category_index[detections['detection_classes'][0]+label_id_offset]['name'])
print("detected probability")
print(detections['detection_scores'][0])
print("true class")
print(IMAGE_PATH)

In [ ]:
!zip -r Tensorflow.zip Tensorflow/
from google.colab import files
files.download("/content/Tensorflow.zip")

In [ ]:
# files.download("/content/Tensorflow.zip")